<a href="https://colab.research.google.com/github/bhagirathtallapragada/Secure-AI-project-phase2/blob/main/multipartysplitlearning_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is an implementation to conduct Split learning using the CIFAR100 dataset with the following settings:
## Cases you need to consider: 
1) Every party has instances of every class, 
2) Every party only holds instances of one class. 

Reporting training and test performance in the above settings with 10, 20, 50, 100 parties. Also report how fast the training converges and time it
take to train with different number of parties involved. 

In [ ]:
# mounting the drive to store the results

from google.colab import drive
drive.mount('/content/drive') #, force_remount = True)

import warnings
warnings.filterwarnings("ignore")

import sys
# sys.path.append()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install syft==0.2.5 --quiet

     |████████████████████████████████| 369 kB 9.0 MB/s 
     |████████████████████████████████| 1.8 MB 46.0 MB/s 
     |████████████████████████████████| 2.1 MB 40.9 MB/s 
     |████████████████████████████████| 484 kB 57.0 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
     |████████████████████████████████| 57 kB 6.2 MB/s 
     |████████████████████████████████| 753.4 MB 7.0 kB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 200 kB 68.7 MB/s 
     |████████████████████████████████| 4.0 MB 43.3 MB/s 
     |████████████████████████████████| 79 kB 8.3 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 58 kB 6.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from time import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.datasets import CIFAR100
import torchvision.transforms as trans
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split,ConcatDataset
from torch import nn, optim
import syft as sy
import time
import torchvision.transforms as transforms
hook = sy.TorchHook(torch)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
train_data = CIFAR100(download=True,root="/content/drive/MyDrive/SPAI_projectphase2/data",transform=transform)
test_data = CIFAR100(root="/content/drive/MyDrive/SPAI_projectphase2/data",train=False,transform=transform)

Files already downloaded and verified


In [ ]:
for image,label in train_data:
    print("Image shape: ",image.shape)
    # print("Image tensor: ", image)
    print("Label: ", label)
    break

Image shape:  torch.Size([3, 32, 32])
Label:  19


In [ ]:
# Number of classes should be 100
len(train_data.classes)

100

## processing the data as needed

In [ ]:
def extract_classes( data, labels,n):
    data_cl=[]
    y=[]
    for i in range(n):
            data_cl.append(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1) ][ : ])
            y.extend(np.full((data_cl[i].shape[0]), i, dtype=int))


    x = np.vstack( (data_cl) )
    y = np.array(y)
    return x, y

In [ ]:
x_train,y_train = extract_classes(train_data.data,np.array(train_data.targets),n=100)
x_train.shape,y_train.shape

((50000, 32, 32, 3), (50000,))

In [ ]:
print(y_train)

train_data.data = x_train
train_data.targets = y_train

[ 0  0  0 ... 99 99 99]


In [ ]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=False)
torch.manual_seed(0)
trainloader.dataset.targets

array([ 0,  0,  0, ..., 99, 99, 99])

## Define a list of total 100 layers that we will be distributing between parties based on number of parties

In [ ]:
total = 100

input_size = 3072
hidden_sizes = [128, 640]
output_size = 100

layers = [nn.Linear(input_size, hidden_sizes[0]), nn.ReLU()]

for i in range(1, int(total/2)-1):
  if (i%2 != 0):
    layers.extend([nn.Linear(hidden_sizes[0], hidden_sizes[1]), nn.ReLU()])
  else:
    layers.extend([nn.Linear(hidden_sizes[1], hidden_sizes[0]), nn.ReLU()])

# the final layer
layers.extend([nn.Linear(hidden_sizes[0], output_size), nn.LogSoftmax(dim=1)])

print(len(layers))
layers

100


[Linear(in_features=3072, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=6

## Defining the splitlearning class to implement multiparty split learning

In [ ]:
class SplitNN:
    def __init__(self, models, optimizers):
        self.models = models
        self.optimizers = optimizers
        
    def forward(self, x):
        a = []
        remote_a = []
        
        a.append(models[0](x))
        if a[-1].location == models[1].location:
            remote_a.append(a[-1].detach().requires_grad_())
        else:
            remote_a.append(a[-1].detach().move(models[1].location).requires_grad_())

        i=1    
        while i < (len(models)-1):
            
            a.append(models[i](remote_a[-1]))
            if a[-1].location == models[i+1].location:
                remote_a.append(a[-1].detach().requires_grad_())
            else:
                remote_a.append(a[-1].detach().move(models[i+1].location).requires_grad_())
            
            i+=1
        
        a.append(models[i](remote_a[-1]))
        self.a = a
        self.remote_a = remote_a
        
        return a[-1]
    
    def backward(self):
        a=self.a
        remote_a=self.remote_a
        optimizers = self.optimizers
        
        i= len(models)-2   
        while i > -1:
            if remote_a[i].location == a[i].location:
                grad_a = remote_a[i].grad.copy()
            else:
                grad_a = remote_a[i].grad.copy().move(a[i].location)
            a[i].backward(grad_a)
            i-=1

    
    def zero_grads(self):
        for opt in optimizers:
            opt.zero_grad()
        
    def step(self):
        for opt in optimizers:
            opt.step()

In [ ]:
def train(x, target, splitNN):
    
    #1) Zero our grads
    splitNN.zero_grads()
    
    #2) Make a prediction
    pred = splitNN.forward(x)
    
    #3) Figure out how much we missed by
    criterion = nn.NLLLoss()
    loss = criterion(pred, target)
  
    #4) Backprop the loss on the end layer
    loss.backward()
    
    #5) Feed Gradients backward through the network
    splitNN.backward()
    
    #6) Change the weights
    splitNN.step()
    
    return loss,pred

In [ ]:
def test(x, target, splitNN):
    
    #1) Zero our grads
    # splitNN.zero_grads()
    
    #2) Make a prediction
    pred = splitNN.forward(x)
    
    #3) Figure out how much we missed by
    # criterion = nn.NLLLoss()
    # loss = criterion(pred, target)
    
    
    return pred

## Multiparty split learning with 10 parties

In [ ]:
n_instances = 10

### creating a list of sequential models by building it from above defined layers list based on number of parties

In [ ]:
models = []
i=0
while(i < 100):
  # each sequential model should have 100/ n_instances layers
  models.append(nn.Sequential(*layers[i:i+int(100/n_instances)]))
  i+=int(100/n_instances)

optimizers = [
    optim.SGD(model.parameters(), lr=0.03,)
    for model in models
]

# This should now show a list of 10 models each with 10 layers
models  

[Sequential(
   (0): Linear(in_features=3072, out_features=128, bias=True)
   (1): ReLU()
   (2): Linear(in_features=128, out_features=640, bias=True)
   (3): ReLU()
   (4): Linear(in_features=640, out_features=128, bias=True)
   (5): ReLU()
   (6): Linear(in_features=128, out_features=640, bias=True)
   (7): ReLU()
   (8): Linear(in_features=640, out_features=128, bias=True)
   (9): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
   (2): Linear(in_features=640, out_features=128, bias=True)
   (3): ReLU()
   (4): Linear(in_features=128, out_features=640, bias=True)
   (5): ReLU()
   (6): Linear(in_features=640, out_features=128, bias=True)
   (7): ReLU()
   (8): Linear(in_features=128, out_features=640, bias=True)
   (9): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
   (2): Linear(in_features=128, out_features=640, bias=True)
   (3): ReLU()
   (4): Linear(in_features=640, out_featur

In [ ]:
alices = []
for i in range(n_instances):
    alices.append(sy.VirtualWorker(hook, id="alice"+str(i+1)))

workers = tuple(alices)
alices

[<VirtualWorker id:alice1 #objects:6>,
 <VirtualWorker id:alice2 #objects:6>,
 <VirtualWorker id:alice3 #objects:6>,
 <VirtualWorker id:alice4 #objects:6>,
 <VirtualWorker id:alice5 #objects:6>,
 <VirtualWorker id:alice6 #objects:6>,
 <VirtualWorker id:alice7 #objects:6>,
 <VirtualWorker id:alice8 #objects:6>,
 <VirtualWorker id:alice9 #objects:6>,
 <VirtualWorker id:alice10 #objects:6>]

In [ ]:
# Send Model Segments to starting locations
model_locations = alices
for model, location in zip(models, model_locations):
    model.send(location)

In [ ]:
splitNN =  SplitNN(models, optimizers)

In [ ]:
models[-1].location.id

'alice10'

In [ ]:
%%time
epochs = 3
for i in range(epochs):
    running_loss = 0
    acc = 0
    for images, labels in trainloader:
        loss = 0
        acc_c = 0
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        loss, pred = train(images, labels, splitNN)

        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        acc_c = train_acc/len(labels)

        running_loss += loss.get()
        acc +=acc_c

    else:
        print("Epoch {} - Training loss: {} Training Accuracy: {}".format(i, running_loss/len(trainloader),acc/len(trainloader)))

Epoch 0 - Training loss: 4.026189804077148 Training Accuracy: 0.07217071611253197
Epoch 1 - Training loss: 3.626110553741455 Training Accuracy: 0.1257992327365729
Epoch 2 - Training loss: 3.262223482131958 Training Accuracy: 0.20076726342711
CPU times: user 9min 59s, sys: 1min 4s, total: 11min 3s
Wall time: 11min 3s
Parser   : 143 ms


In [ ]:
testloader = torch.utils.data.DataLoader(test_data,shuffle=False)
torch.manual_seed(0)

In [ ]:
%%time
# loss=0
acc_c = 0
acc = 0
with torch.no_grad():
  for images, labels in testloader:
    images = images.send(models[0].location)
    images = images.view(images.shape[0], -1)
    labels = labels.send(models[-1].location)
    pred = test(images, labels, splitNN)
    pred = torch.argmax(pred, dim=1)
    train_acc = torch.sum(pred == labels).get().item()
    # acc_c = train_acc/len(labels)
    # print(images.shape,labels.shape)
    acc +=train_acc
    # running_loss += loss.get()
print("Testing accuracy: {}".format(acc/len(testloader)))

Testing accuracy: 0.01
CPU times: user 9min 13s, sys: 47.2 s, total: 10min
Wall time: 9min 59s


## Multiparty split learning with 20 parties (may need to restart runtime before implementing this section)

In [ ]:
n_instances = 20

### creating a list of sequential models by building it from above defined layers list based on number of parties

In [ ]:
models = []
i=0
while(i < 100):
  # each sequential model should have 100/ n_instances layers
  models.append(nn.Sequential(*layers[i:i+int(100/n_instances)]))
  i+=int(100/n_instances)

optimizers = [
    optim.SGD(model.parameters(), lr=0.03,)
    for model in models
]

# This should now show a list of 20 models each with 5 layers
models  

[Sequential(
   (0): Linear(in_features=3072, out_features=128, bias=True)
   (1): ReLU()
   (2): Linear(in_features=128, out_features=640, bias=True)
   (3): ReLU()
   (4): Linear(in_features=640, out_features=128, bias=True)
 ), Sequential(
   (0): ReLU()
   (1): Linear(in_features=128, out_features=640, bias=True)
   (2): ReLU()
   (3): Linear(in_features=640, out_features=128, bias=True)
   (4): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
   (2): Linear(in_features=640, out_features=128, bias=True)
   (3): ReLU()
   (4): Linear(in_features=128, out_features=640, bias=True)
 ), Sequential(
   (0): ReLU()
   (1): Linear(in_features=640, out_features=128, bias=True)
   (2): ReLU()
   (3): Linear(in_features=128, out_features=640, bias=True)
   (4): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
   (2): Linear(in_features=128, out_features=640, bias=True)
   (3): ReLU()
   (4): Li

In [ ]:
alices = []
for i in range(n_instances):
    alices.append(sy.VirtualWorker(hook, id="alice"+str(i+1)))

workers = tuple(alices)
alices

[<VirtualWorker id:alice1 #objects:6>,
 <VirtualWorker id:alice2 #objects:6>,
 <VirtualWorker id:alice3 #objects:6>,
 <VirtualWorker id:alice4 #objects:6>,
 <VirtualWorker id:alice5 #objects:6>,
 <VirtualWorker id:alice6 #objects:6>,
 <VirtualWorker id:alice7 #objects:6>,
 <VirtualWorker id:alice8 #objects:6>,
 <VirtualWorker id:alice9 #objects:6>,
 <VirtualWorker id:alice10 #objects:6>,
 <VirtualWorker id:alice11 #objects:6>,
 <VirtualWorker id:alice12 #objects:6>,
 <VirtualWorker id:alice13 #objects:6>,
 <VirtualWorker id:alice14 #objects:6>,
 <VirtualWorker id:alice15 #objects:6>,
 <VirtualWorker id:alice16 #objects:6>,
 <VirtualWorker id:alice17 #objects:6>,
 <VirtualWorker id:alice18 #objects:6>,
 <VirtualWorker id:alice19 #objects:6>,
 <VirtualWorker id:alice20 #objects:6>]

In [ ]:
# Send Model Segments to starting locations
model_locations = alices
for model, location in zip(models, model_locations):
    model.send(location)

In [ ]:
splitNN =  SplitNN(models, optimizers)

In [ ]:
models[-1].location.id

'alice20'

In [ ]:
%%time
epochs = 3
for i in range(epochs):
    running_loss = 0
    acc = 0
    for images, labels in trainloader:
        loss = 0
        acc_c = 0
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        loss, pred = train(images, labels, splitNN)

        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        acc_c = train_acc/len(labels)

        running_loss += loss.get()
        acc +=acc_c

    else:
        print("Epoch {} - Training loss: {} Training Accuracy: {}".format(i, running_loss/len(trainloader),acc/len(trainloader)))

Epoch 0 - Training loss: 4.026189804077148 Training Accuracy: 0.07217071611253197
Epoch 1 - Training loss: 3.626110553741455 Training Accuracy: 0.1257992327365729
Epoch 2 - Training loss: 3.262223482131958 Training Accuracy: 0.20076726342711
CPU times: user 11min 24s, sys: 1min 45s, total: 13min 10s
Wall time: 13min 11s


In [ ]:
testloader = torch.utils.data.DataLoader(test_data,shuffle=False)
torch.manual_seed(0)

In [ ]:
%%time
# loss=0
acc_c = 0
acc = 0
with torch.no_grad():
  for images, labels in testloader:
    images = images.send(models[0].location)
    images = images.view(images.shape[0], -1)
    labels = labels.send(models[-1].location)
    pred = test(images, labels, splitNN)
    pred = torch.argmax(pred, dim=1)
    train_acc = torch.sum(pred == labels).get().item()
    # acc_c = train_acc/len(labels)
    # print(images.shape,labels.shape)
    acc +=train_acc
    # running_loss += loss.get()
print("Testing accuracy: {}".format(acc/len(testloader)))

Testing accuracy: 0.01
CPU times: user 11min 21s, sys: 1min 17s, total: 12min 39s
Wall time: 12min 37s


## Multiparty split learning with 50 parties

In [ ]:
n_instances = 50

### creating a list of sequential models by building it from above defined layers list based on number of parties

In [ ]:
models = []
i=0
while(i < 100):
  # each sequential model should have 100/ n_instances layers
  models.append(nn.Sequential(*layers[i:i+int(100/n_instances)]))
  i+=int(100/n_instances)

optimizers = [
    optim.SGD(model.parameters(), lr=0.03,)
    for model in models
]

# This should now show a list of 50 models each with 2 layers
models  

[Sequential(
   (0): Linear(in_features=3072, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1

In [ ]:
alices = []
for i in range(n_instances):
    alices.append(sy.VirtualWorker(hook, id="alice"+str(i+1)))

workers = tuple(alices)
alices

[<VirtualWorker id:alice1 #objects:6>,
 <VirtualWorker id:alice2 #objects:6>,
 <VirtualWorker id:alice3 #objects:6>,
 <VirtualWorker id:alice4 #objects:6>,
 <VirtualWorker id:alice5 #objects:6>,
 <VirtualWorker id:alice6 #objects:6>,
 <VirtualWorker id:alice7 #objects:6>,
 <VirtualWorker id:alice8 #objects:6>,
 <VirtualWorker id:alice9 #objects:6>,
 <VirtualWorker id:alice10 #objects:6>,
 <VirtualWorker id:alice11 #objects:6>,
 <VirtualWorker id:alice12 #objects:6>,
 <VirtualWorker id:alice13 #objects:6>,
 <VirtualWorker id:alice14 #objects:6>,
 <VirtualWorker id:alice15 #objects:6>,
 <VirtualWorker id:alice16 #objects:6>,
 <VirtualWorker id:alice17 #objects:6>,
 <VirtualWorker id:alice18 #objects:6>,
 <VirtualWorker id:alice19 #objects:6>,
 <VirtualWorker id:alice20 #objects:6>,
 <VirtualWorker id:alice21 #objects:6>,
 <VirtualWorker id:alice22 #objects:6>,
 <VirtualWorker id:alice23 #objects:6>,
 <VirtualWorker id:alice24 #objects:6>,
 <VirtualWorker id:alice25 #objects:6>,
 <Virtual

In [ ]:
# Send Model Segments to starting locations
model_locations = alices
for model, location in zip(models, model_locations):
    model.send(location)

In [ ]:
splitNN =  SplitNN(models, optimizers)

In [ ]:
%%time
epochs = 3
for i in range(epochs):
    running_loss = 0
    acc = 0
    for images, labels in trainloader:
        loss = 0
        acc_c = 0
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        loss,pred = train(images, labels, splitNN)
        # print(pred.shape)
        # print(labels.shape)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        acc_c = train_acc/len(labels)
        # print(train_acc.get()) 
        running_loss += loss.get()
        acc +=acc_c

    else:
        print("Epoch {} - Training loss: {} Training Accuracy: {}".format(i, running_loss/len(trainloader),acc/len(trainloader)))

Epoch 0 - Training loss: 4.026189804077148 Training Accuracy: 0.07217071611253197
Epoch 1 - Training loss: 3.626110553741455 Training Accuracy: 0.1257992327365729
Epoch 2 - Training loss: 3.262223482131958 Training Accuracy: 0.20076726342711
CPU times: user 16min 9s, sys: 5min 20s, total: 21min 29s
Wall time: 21min 29s


In [ ]:
testloader = torch.utils.data.DataLoader(test_data,shuffle=False)
torch.manual_seed(0)

In [ ]:
%%time
# loss=0
acc_c = 0
acc = 0
with torch.no_grad():
  for images, labels in testloader:
    images = images.send(models[0].location)
    images = images.view(images.shape[0], -1)
    labels = labels.send(models[-1].location)
    pred = test(images, labels, splitNN)
    pred = torch.argmax(pred, dim=1)
    train_acc = torch.sum(pred == labels).get().item()
    # acc_c = train_acc/len(labels)
    # print(images.shape,labels.shape)
    acc +=train_acc
    # running_loss += loss.get()
print("Testing accuracy: {}".format(acc/len(testloader)))

Testing accuracy: 0.01
CPU times: user 25min 7s, sys: 41min 30s, total: 1h 6min 38s
Wall time: 1h 6min 35s


## Multiparty split learning with 100 parties

In [ ]:
n_instances = 100

### creating a list of sequential models by building it from above defined layers list based on number of parties

In [ ]:
epochs = 3

# Define our model segments
models = []
input_size = 3072
hidden_sizes = [128, 640]
output_size = 100
inp = input_size
out = hidden_sizes[0]
k=0
for i in range(99):

  models.append(nn.Sequential(
              nn.Linear(inp, out),
              nn.ReLU(),
  ))
  inp = hidden_sizes[k]
  if k==0:
      k=1
  else:
      k=0
  out = hidden_sizes[k]

models.append(nn.Sequential(
                nn.Linear(inp, output_size),
                nn.LogSoftmax(dim=1)
    ))
models

[Sequential(
   (0): Linear(in_features=3072, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1

In [ ]:
# models = []
# i=0
# while(i < 100):
#   # each sequential model should have 100/ n_instances layers
#   models.append(nn.Sequential(*layers[i:i+(100/n_instances)]))
#   i+=n_instances

optimizers = [
    optim.SGD(model.parameters(), lr=0.03,)
    for model in models
]

# # This should now show a list of 100 models each with 1 layers
# models  

In [ ]:
len(models)

100

In [ ]:
alices = []
for i in range(n_instances):
    alices.append(sy.VirtualWorker(hook, id="alice"+str(i+1)))

workers = tuple(alices)
alices

[<VirtualWorker id:alice1 #objects:6>,
 <VirtualWorker id:alice2 #objects:6>,
 <VirtualWorker id:alice3 #objects:6>,
 <VirtualWorker id:alice4 #objects:6>,
 <VirtualWorker id:alice5 #objects:6>,
 <VirtualWorker id:alice6 #objects:6>,
 <VirtualWorker id:alice7 #objects:6>,
 <VirtualWorker id:alice8 #objects:6>,
 <VirtualWorker id:alice9 #objects:6>,
 <VirtualWorker id:alice10 #objects:6>,
 <VirtualWorker id:alice11 #objects:6>,
 <VirtualWorker id:alice12 #objects:6>,
 <VirtualWorker id:alice13 #objects:6>,
 <VirtualWorker id:alice14 #objects:6>,
 <VirtualWorker id:alice15 #objects:6>,
 <VirtualWorker id:alice16 #objects:6>,
 <VirtualWorker id:alice17 #objects:6>,
 <VirtualWorker id:alice18 #objects:6>,
 <VirtualWorker id:alice19 #objects:6>,
 <VirtualWorker id:alice20 #objects:6>,
 <VirtualWorker id:alice21 #objects:6>,
 <VirtualWorker id:alice22 #objects:6>,
 <VirtualWorker id:alice23 #objects:6>,
 <VirtualWorker id:alice24 #objects:6>,
 <VirtualWorker id:alice25 #objects:6>,
 <Virtual

In [ ]:
# Send Model Segments to starting locations
model_locations = alices
for model, location in zip(models, model_locations):
    model.send(location)

In [ ]:
splitNN =  SplitNN(models, optimizers)

In [ ]:
epochs = 3
for i in range(epochs):
    running_loss = 0
    acc = 0
    for images, labels in trainloader:
        loss = 0
        acc_c = 0
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        loss,pred = train(images, labels, splitNN)
        # print(pred.shape)
        # print(labels.shape)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        acc_c = train_acc/len(labels)
        # print(train_acc.get()) 
        running_loss += loss.get()
        acc +=acc_c

    else:
        print("Epoch {} - Training loss: {} Training Accuracy: {}".format(i, running_loss/len(trainloader),acc/len(trainloader)))

Epoch 0 - Training loss: 4.175104141235352 Training Accuracy: 0.050831202046035806
Epoch 1 - Training loss: 3.870326042175293 Training Accuracy: 0.08104219948849105
Epoch 2 - Training loss: 3.449857234954834 Training Accuracy: 0.15361253196930946


In [ ]:
testloader = torch.utils.data.DataLoader(test_data,shuffle=False)
torch.manual_seed(0)

In [ ]:
%%time
# loss=0
acc_c = 0
acc = 0
with torch.no_grad():
  for images, labels in testloader:
    images = images.send(models[0].location)
    images = images.view(images.shape[0], -1)
    labels = labels.send(models[-1].location)
    pred = test(images, labels, splitNN)
    pred = torch.argmax(pred, dim=1)
    train_acc = torch.sum(pred == labels).get().item()
    # acc_c = train_acc/len(labels)
    # print(images.shape,labels.shape)
    acc +=train_acc
    # running_loss += loss.get()
print("Testing accuracy: {}".format(acc/len(testloader)))

Testing accuracy: 0.01
